In [1]:
import pandas as pd
import numpy as np

# import training and test datasets {-}

In [116]:
# pull in files and create dataframes
train = np.loadtxt("pa3train.txt")
test = np.loadtxt("pa3test.txt")



# training data 
df_train = pd.DataFrame(train)
df_train_2 = df_train[df_train.apply(lambda x: x[819] == 1 or x[819] == 2, axis=1)]
#df_train_2 = df_train_2.sample(frac=1)
# test data
df_test = pd.DataFrame(test)
df_test_2 = df_test[df_test.apply(lambda x: x[819] == 1 or x[819] == 2, axis=1)]
#df_dict = pd.DataFrame("pa3dictionary.txt")



X_train = df_train.drop([819], axis=1)
y_train = df_train[819]

X_train_2 = df_train_2.drop([819], axis=1)
y_train_2 = df_train_2[819]

## mapping 1 -> 1 and 2 -> -1
y_train_2_trans = y_train_2.apply(lambda x: 1 if x > 1 else -1)

X_test = df_test.drop([819], axis=1)
y_test = df_test[819]

X_test_2 = df_test_2.drop([819], axis=1)
y_test_2 = df_test_2[819]
# mapping 1 -> 1 and 2 -> -1
y_test_2_trans = y_test_2.apply(lambda x: 1 if x > 1 else -1)

label = 819

In [122]:
len(y_test_2_trans)

377

# Create Perceptron Class to train classifier {-}

In [130]:
class Perceptron:
    def __init__(self, rounds):
        self.rounds = rounds
        self.X = None
        self.y = None
        self.w = None
        self.w_avg = None
        self.array_of_w = []
        self.predictions = []
        self.voted_predictions = []
        self.avg_predictions = []
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.w = pd.Series(np.arange(len(self.X.columns)))*0
        m = 1
        c = 1
        for j in range(self.rounds):
            for i in range(len(self.X)):
                if self.y.iloc[i]*self.w.dot(self.X.iloc[i]) <= 0:
                    self.w = self.w + self.y.iloc[i]*self.X.iloc[i]
                    w = self.w
                    m = m + 1 
                    c = 1
                    self.array_of_w.append([w, c])
                else: 
                    c = c + 1
                    index = len(self.array_of_w) - 1 # last position of last [w, c]
                    self.array_of_w[index][1] = c
    
    def perceptron_get_sign(self, w, x):
        return -1 if w.dot(x) <= 0 else 1
    
    def perceptron_get_sign_with_c(self, w, x, c):
        return c*self.perceptron_get_sign(w,x)
        
    def predict(self, X):
        self.predictions = []
        for i in range(len(X)):
            x = X.iloc[i]
            self.predictions.append(self.perceptron_get_sign(self.w, x))
        return self.predictions
    
    def voted_predict(self, X):
        self.voted_predictions = []
        for i in range(len(X)):
            x = X.iloc[i]
            voted_sum = 0
            for w_c in self.array_of_w:
                w = w_c[0]
                c = w_c[1]
                voted_sum = voted_sum + self.perceptron_get_sign_with_c(w, x, c)

            pred = -1 if voted_sum <= 0 else 1
            self.voted_predictions.append(pred)

        return self.voted_predictions
    
    def avg_predict(self, X):
        self.avg_predictions = []
        # get first w,c pair
        w = self.array_of_w[0][0] # grab w
        c = self.array_of_w[0][1] # grab c
        w_avg = c*w # [0, ..., 0] + first c*w
        for i in range(1, len(self.array_of_w)):
                w = self.array_of_w[i][0]
                c = self.array_of_w[i][1]
                w_avg = w_avg + c*w

        for i in range(len(X)):
            x = X.iloc[i]
            sign = self.perceptron_get_sign(w_avg, x)
            self.avg_predictions.append(sign)
        
        self.w_avg = w_avg

        return self.avg_predictions
        
        

In [131]:
def getError(preds, y):
    return 1 - ((preds == y).sum() / len(y))

# Question 1: {-}
    What are the training errors and the test errors of perceptron, voted perceptron and averaged perceptron after two, three and four passes?
### First, will create an instances of Perceptrons with 2, 3, and 4 rounds {-}
### Then fit the training data to each one of these {-}
##  The y labels have been mapped 1 -> -1 and 2 -> 1 {-}

In [179]:
# takes one argument for the number of rounds 

# create for train set
cls2 = Perceptron(2)
cls3 = Perceptron(3)
cls4 = Perceptron(4)


In [180]:
cls2.fit(X_train_2, y_train_2_trans)
cls3.fit(X_train_2, y_train_2_trans)
cls4.fit(X_train_2, y_train_2_trans)


## Get predictions for Perceptron and compute errors  {-}

In [181]:
# predict for training data
train_preds2 = cls2.predict(X_train_2)
train_preds3 = cls3.predict(X_train_2)
train_preds4 = cls4.predict(X_train_2)


# predict for test data
test_preds2 = cls2.predict(X_test_2)
test_preds3 = cls3.predict(X_test_2)
test_preds4 = cls4.predict(X_test_2)



In [182]:
print("Errors on Training data for Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(train_preds2, y_train_2_trans), getError(train_preds3, y_train_2_trans), getError(train_preds4, y_train_2_trans)))

print("\n\nErrors on Test data for Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(test_preds2, y_test_2_trans), getError(test_preds3, y_test_2_trans), getError(test_preds4, y_test_2_trans)))



Errors on Training data for Perceptron:
Error for 2 rounds: 0.0357798165137615
Error for 3 rounds: 0.01834862385321101
Error for 4 rounds: 0.016513761467889854


Errors on Test data for Perceptron:
Error for 2 rounds: 0.06100795755968169
Error for 3 rounds: 0.04509283819628651
Error for 4 rounds: 0.04509283819628651


# Get predictions for voted perceptron and compute errors {-}

In [136]:
# predict voted perceptron on training data
train_voted_preds2 = cls2.voted_predict(X_train_2)
train_voted_preds3 = cls3.voted_predict(X_train_2)
train_voted_preds4 = cls4.voted_predict(X_train_2)



# predict voted perceptron on test data
test_voted_preds2 = cls2.voted_predict(X_test_2)
test_voted_preds3 = cls3.voted_predict(X_test_2)
test_voted_preds4 = cls4.voted_predict(X_test_2)

In [137]:
print("Errors on Training Data for Voted Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(train_voted_preds2, y_train_2_trans), getError(train_voted_preds3, y_train_2_trans), getError(train_voted_preds4, y_train_2_trans)))



print("\n\nErrors on Test Data for Voted Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(test_voted_preds2, y_test_2_trans), getError(test_voted_preds3, y_test_2_trans), getError(test_voted_preds4, y_test_2_trans)))



Errors on Training Data for Voted Perceptron:
Error for 2 rounds: 0.038532110091743066
Error for 3 rounds: 0.026605504587155937
Error for 4 rounds: 0.022018348623853212


Errors on Test Data for Voted Perceptron:
Error for 2 rounds: 0.06100795755968169
Error for 3 rounds: 0.04244031830238726
Error for 4 rounds: 0.04509283819628651


### Get predictions for averaged perceptron and compute errors {-}

In [138]:
train_avg_preds2 = cls2.avg_predict(X_train_2)
train_avg_preds3 = cls3.avg_predict(X_train_2)
train_avg_preds4 = cls4.avg_predict(X_train_2)


test_avg_preds2 = cls2.avg_predict(X_test_2)
test_avg_preds3 = cls3.avg_predict(X_test_2)
test_avg_preds4 = cls4.avg_predict(X_test_2)

In [139]:
print("Errors for Averaged Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(train_avg_preds2, y_train_2_trans), getError(train_avg_preds3, y_train_2_trans), getError(train_avg_preds4, y_train_2_trans)))


print("\n\nErrors for Averaged Perceptron:\nError for 2 rounds: {}\nError for 3 rounds: {}\nError for 4 rounds: {}"
      .format(getError(test_avg_preds2, y_test_2_trans), getError(test_avg_preds3, y_test_2_trans), getError(test_avg_preds4, y_test_2_trans)))



Errors for Averaged Perceptron:
Error for 2 rounds: 0.05137614678899083
Error for 3 rounds: 0.034862385321100864
Error for 4 rounds: 0.031192660550458662


Errors for Averaged Perceptron:
Error for 2 rounds: 0.08222811671087538
Error for 3 rounds: 0.06100795755968169
Error for 4 rounds: 0.050397877984084904


## Question 2: {-}
    Consider the averaged perceptron classifier w_avg that you built 
    by running three passes on the data. We will now try to interpret 
    this classifier.
    
    Find the three coordinates in w_avg with the highest and lowest 
    values. What are the words (from pa3dictionary.txt) that 
    correspond to these coordinates?

In [175]:
train_largest = list(cls3.w_avg.sort_values(ascending=False).index[:3])
train_smallest = list(cls3.w_avg.sort_values().index[:3])

In [176]:
dictionary = []
with open("pa3dictionary.txt", "r") as f:
    dictionary = f.read().split()

a
a True
a True
a
a True
a True
b
b True
b True
c
c True
d
d True


In [177]:
string = "Words with highest values are:\n"
adjectives = ["Highest", "Second Highest", "Third Highest"]
i = 0
for index in train_largest:
    string += adjectives[i] + ": " + dictionary[index] + "\n"
    i+=1
    
print("Coordinates of largest in w_avg are: {}".format(train_largest))
print(string)


Coordinates of largest in w_avg are: [78, 469, 393]
Words with highest values are:
Highest: he
Second Highest: team
Third Highest: game



In [178]:
string = "Words with smallest values are:\n"
adjectives = ["Least", "Second least", "Third least"]
i = 0
for index in train_smallest:
    string += adjectives[i] + ": " + dictionary[index] + "\n"
    i+=1

print("Coordinates of smallest in w_avg are: {}".format(train_smallest))
print(string)

Coordinates of smallest in w_avg are: [438, 466, 203]
Words with smallest values are:
Least: file
Second least: program
Third least: line



### Question 3:  {-}
    Given a test example x, the one-vs-all classifier predicts as follows. If Ci(x) = i for exactly one i = 1,...,6, 
    then predict label i. If Ci(x) = i for more than one i in 1,...,6, or if Ci(x) = i for no i, then report Don’t 
    Know
    
### I have used the value 7 to report don't know, which is also row 6 in the confusion matrix {-}

In [145]:
train = np.loadtxt("pa3train.txt")
test = np.loadtxt("pa3test.txt")



# training data 
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

X_train = df_train.drop([819], axis=1)
y_train = df_train[819]

# create array of labels where each array is 1 for i and -1 for not i where i is 1, 2, 3, ..., 6
y_trains_1_to_6 = []
for i in range(6):
    matchNum = i + 1
    y_trains_1_to_6.append(y_train.apply(lambda x: 1 if x == matchNum else -1))
    

X_test = df_test.drop([819], axis=1)
y_test = df_test[819]
y_test_1_to_6 = []
for i in range(6):
    matchNum = i + 1
    y_test_1_to_6.append(y_test.apply(lambda x: 1 if x == matchNum else -1))

In [144]:
len(y_trains_1_to_6), len(y_test_1_to_6)

(6, 6)

In [146]:
classifiers = []
for i in range(len(y_trains_1_to_6)):
    cls = Perceptron(1)
    cls.fit(X_train, y_trains_1_to_6[i])
    classifiers.append(cls)

In [147]:
classes = {
    0: 1, 
    1: 2, 
    2: 3, 
    3: 4, 
    4: 5, 
    5: 6
}

# get predictions on test data and build confusion matrix
preds = []
for i in range(len(X_test)):
    x = X_test.iloc[i]
    arr = []
    the_class_value = None
    for j in range(len(classifiers)):
        classifier_w = classifiers[j].w
        sign = 0 if classifier_w.dot(x) <= 0 else 1
        if sign == 1:
            the_class_value = classes[j]
        arr.append(sign)
    arr = pd.Series(arr)
    
    if arr.sum() == 1: 
        preds.append(the_class_value)
    elif arr.sum() > 1:
        preds.append(7)
    elif arr.sum() <= 0:
        preds.append(7)
    
preds = pd.Series(preds)
confusion = pd.DataFrame(0, columns=np.arange(6), index=np.arange(7))

In [148]:
# for purpose of matrix i = 1, 2, 3, 4, 5, 6 will map to i = 0,1,2,3,4,5
for i in range(len(preds)):
    confusion.iloc[preds[i] - 1][y_test[i] - 1] += 1

In [149]:
# divide each j the number of j test examples
for i in range(6):
    confusion[i] = confusion[i] / y_test.value_counts()[i+1]

### Here is confusion matrix on test data using the multiclass classifier built on training data {-}
#### The index 0, 1, 2, 3, 4, 5 map to i = 1, 2, 3, 4, 5, 6, respectively. There is a 7th row, which is the 'I don't know' row {-}

In [174]:
confusion 

,0,1,2,3,4,5
0,0.718919,0.005208,0.034286,0.021739,0.000000,0.000000
1,0.010811,0.656250,0.034286,0.027174,0.012821,0.018519
2,0.000000,0.015625,0.371429,0.000000,0.000000,0.027778
3,0.016216,0.005208,0.000000,0.690217,0.000000,0.000000
4,0.016216,0.031250,0.074286,0.005435,0.801282,0.120370
5,0.005405,0.010417,0.034286,0.000000,0.070513,0.490741
6,0.232432,0.276042,0.451429,0.255435,0.115385,0.342593


## Question and Answers to Q3 {-}
#### Looking at the confusion matrix, what are the i and j in the following statements? {-}
  ##### (a) The perceptron classifier has the highest accuracy for examples that belong to class i. {-}
        Answer: Classifier i = 5 (index 4) has the highest accuracy @ .801282 
  ##### (b) The perceptron classifier has the least accuracy for examples that belong to class i. {-}
        Answer: Classifier i = 3 (index 2) has the least accuracy @ .371429
  ##### (c) The perceptron classifier most often mistakenly classifies an example in class j as belonging to class i, for i, j ∈ {1, 2, 3, 4, 5, 6} (i.e., excluding Don’t Know). {-}
        Answer: Classiefier i = 5 (index 4) most often mistakenly classifies as 6 (column 5) @ .120370